<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/AddestramentoScanner/Creazione_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [1]:
#Librerie
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, TensorDataset
import time
import os
import shutil
import copy
from PIL import Image

##Check device


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

1.9.0+cu102
True
Tesla K80
cuda:0


##Def variabili

In [3]:
#scanner
scanner_name = 'DigitalPersona'

#dataset
pathDataset=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}'

pathCrop=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}_patch/'

##Collegamento google drive

In [4]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**Dataset**

##Creazione patch

In [9]:
import cv2

flag=False
Directory = os.listdir(pathDataset)
for classe in Directory:
  classes_path = os.listdir(pathDataset+"/" + classe)  
  pathSave=pathCrop+classe
  print("working on: "+classe)
  for input in classes_path:
    #print('Working on '+ input)
    if input.endswith('.png'):  
      img = Image.open(pathDataset+"/"+classe+'/' + input)
      n=img.size
      n_mod=calc_size(n)
      crop_transforms=transforms.TenCrop((n_mod[1],n_mod[0])).to(device)
      
      crops=crop_transforms(img) #contiene le 10 patch
      i=0
      for crop in crops:
        i+=1
        name=input.split('.')[0]+"_patch"+str(i)+".png"
        if(!cv2.imwrite(pathCrop+classe+"/"+name, crop)): 
          print("error saving")
          flag=True
          break
        #crop.save(pathCrop+classe+"/"+name)
      if flag:break  
  if flag: break        
  print("Finish: "+classe+". Salvate "+len(os.listdir(pathCrop+"/" + classe))+" immagini")     
  

working on: Live
Finish classe: Live


In [8]:
def calc_size(n):
  return tuple(int(np.ceil(i * (80/100))) for i in n)

check

In [11]:
dim_before=0
dim_after=0
for classe in ["Live","Spoof"]:
  dim_before+=(len(os.listdir(pathDataset+"/" + classe)))
  dim_after+=(len(os.listdir(pathCrop+"/" + classe)))
print(dim_before,dim_after)  
if dim_after==dim_before*10: print("OK")
else: print("BAD")

2000 20000
OK
